# Importing libraries, Examining data

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
from pathlib import Path
import sys
from patsy import dmatrices
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [2]:
# Read csv
originaldata = pd.read_csv('https://raw.githubusercontent.com/Bomsk/DA3_2023/main/Assignment%202/listings.csv')

# Drop rows with missing prices
data = originaldata.dropna(subset=['price'])

# Clean price column
data["price"] = data["price"].replace('[\$,]', '', regex=True).astype(float)

# Filter
data = data[(data['accommodates'] >= 2) &            # 'accommodates' for 2-6 people
            (data['accommodates'] <= 6) &            # 'accommodates' for 2-6 people   
            (data['room_type'] != 'Hotel room') &    # filter out 'Hotel room'
            (data['room_type'] != 'Shared room')]    # filter out 'Shared room'

data

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,197677,https://www.airbnb.com/rooms/197677,20231227173752,2023-12-28,city scrape,Rental unit in Sumida · ★4.77 · 1 bedroom · 2 ...,NaN,NaN,https://a0.muscache.com/pictures/38437056/d27f...,964081,...,4.83,4.54,4.79,M130003350,f,1,1,0,0,1.16
2,905944,https://www.airbnb.com/rooms/905944,20231227173752,2023-12-28,city scrape,Rental unit in Shibuya · ★4.78 · 2 bedrooms · ...,NaN,Hatagaya is a great neighborhood located 4 min...,https://a0.muscache.com/pictures/miso/Hosting-...,4847803,...,4.91,4.78,4.78,Hotels and Inns Business Act | 渋谷区保健所長 | 31渋健生...,t,5,5,0,0,1.52
5,1196177,https://www.airbnb.com/rooms/1196177,20231227173752,2023-12-28,city scrape,Home in 足立区 · ★4.75 · 1 bedroom · 1.5 shared b...,NaN,There are shopping mall near Senjuohashi stati...,https://a0.muscache.com/pictures/72890882/05ec...,5686404,...,4.90,4.70,4.78,M130007760,f,1,0,1,0,0.85
6,3845889,https://www.airbnb.com/rooms/3845889,20231227173752,2023-12-27,city scrape,Rental unit in Suginami-ku · ★4.86 · 1 bedroom...,NaN,Koenji town is very unique and stimulating pla...,https://a0.muscache.com/pictures/e498a703-e523...,19089769,...,4.96,4.90,4.87,M130000777,f,2,2,0,0,0.97
7,3846001,https://www.airbnb.com/rooms/3846001,20231227173752,2023-12-27,city scrape,Rental unit in Suginami-ku · ★4.92 · Studio · ...,NaN,NaN,https://a0.muscache.com/pictures/d697e82d-cb99...,19089769,...,4.95,4.90,4.85,M130008732,f,2,2,0,0,1.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13717,1052523990478131414,https://www.airbnb.com/rooms/1052523990478131414,20231227173752,2023-12-28,city scrape,Rental unit in Katsushika City · ★New · 2 bedr...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,552216877,...,NaN,NaN,NaN,M130035708,t,2,2,0,0,NaN
13718,1052570322244321995,https://www.airbnb.com/rooms/1052570322244321995,20231227173752,2023-12-28,city scrape,Home in Shinagawa City · ★New · 1 bedroom · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,28621978,...,NaN,NaN,NaN,M130025425,f,7,0,7,0,NaN
13719,1052573121521757761,https://www.airbnb.com/rooms/1052573121521757761,20231227173752,2023-12-28,city scrape,Home in Shinagawa City · ★New · 1 bedroom · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,28621978,...,NaN,NaN,NaN,M130025425,f,7,0,7,0,NaN
13721,1052618956131641134,https://www.airbnb.com/rooms/1052618956131641134,20231227173752,2023-12-28,city scrape,Rental unit in Taito City · ★New · 2 bedrooms ...,NaN,Ichiyu Tokyo Apartment is near Iriya and Asaku...,https://a0.muscache.com/pictures/miso/Hosting-...,292115979,...,NaN,NaN,NaN,M130019957,f,47,45,2,0,NaN
